# Algoritmo genetico para obtener la mejor combinacion de medicamentos dada una patologia



** Setear los parametros del individuo (background, sintomas, efectos secundarios) **

In [6]:
class PathologyIndividual:
    "Clase destinada a las patologias, antecedentes del paciente, sintomas de la patologia y efectos de los medicamentos "

    def __init__(self, pathology, diccBackground, diccMedicines, arrSymtopm, arrWeigth, arrEffects, bayesianNetwork, population):
        self.pathology = pathology
        self.diccBackground  = diccBackground[:]
        self.diccMedicines   = diccMedicines[:]
        self.arrSymtopm      = arrSymtopm[:]
        self.arrWeigth       = arrWeigth[:]
        self.arrEffects      = arrEffects[:]
        self.bayesianNetwork = bayesianNetwork
        self.population      = population[:]


<b>Clase abstracta del individuo (medicamentos)</b>

In [ ]:
class Individuo:
    "Clase abstracta para individuos de un algoritmo evolutivo."

    def __init__(self, chromosome):
        self.chromosome = chromosome

    def crossover(self, other):
        "Retorna un nuevo individuo cruzando self y other."
        raise NotImplementedError
        
    def mutate(self):
        "Cambia los valores de algunos genes."
        raise NotImplementedError        

<b>Clase concreta de un individuo del problema de las n-reinas</b>

In [ ]:
import random

class Individual_medicines(Individual):
    "Clase que implementa el individuo en el problema de la busqueda de medicamentos que maximice la ausencia de los sintomas."

    def __init__(self, chromosome):
        self.chromosome = chromosome[:]
        self.fitness = -1

    def crossover_onepoint(self, other):
        "Retorna dos nuevos individuos del cruzamiento de un punto entre self y other "
        c = random.randrange(len(self.chromosome))
        ind1 = Individual_medicines(self.chromosome[:c] + other.chromosome[c:])
        ind2 = Individual_medicines(other.chromosome[:c] + self.chromosome[c:])
        return [ind1, ind2]   
    
    def crossover_uniform(self, other):
        chromosome1 = []
        chromosome2 = []
        "Retorna dos nuevos individuos del cruzamiento uniforme entre self y other "
        for i in range(len(self.chromosome)):
            if random.uniform(0, 1) < 0.5:
                chromosome1.append(self.chromosome[i])
                chromosome2.append(other.chromosome[i])
            else:
                chromosome1.append(other.chromosome[i])
                chromosome2.append(self.chromosome[i])
        ind1 = Individual_medicines(chromosome1)
        ind2 = Individual_medicines(chromosome2)
        return [ind1, ind2]     

    def mutate_position(self):
        "Cambia aleatoriamente el estado del medicamento (prescribir, prescribir bajas condiciones, no prescribir)"
        
        for i in range(len(self.chromosome))
        mutated_ind = Individual_nqueens(self.chromosome[:])
        indexPos = random.randint(0, len(mutated_ind.chromosome)-1)
        newPos = random.randint(0, len(mutated_ind.chromosome)-1)
        mutated_ind.chromosome[indexPos] = newPos
        return mutated_ind
    

<b>Funcion de fitness para evaluar un individuo del problema de combinacion de medicamentos</b>

In [ ]:
def setStatesChromosome(diccMedicines, chromosome):
    n = len(chromosome)
    i = 0
    for key in diccMedicines:
        diccMedicines[key] = chromosome[i]
        i+=1
    return diccMedicines
        

In [10]:
def fitnessFunction(pathologyInd, chromosome):
    """Retorna el fitness de un cromosoma en el problema de combinacion de medicamentos """
    bp = BeliefPropagation(pathologyInd.bayesianNetwork)
    mergeSymEff      = pathologyInd.arrSymtopm + pathologyInd.arrEfects    
    newDiccMedicines = setStates(pathologyInd.diccMedicines, chromosome)   
    evidence = createEvidence(pathologyInd.diccBackground,pathologyInd.diccMedicines)   
    phi      = bp.query(variables=mergeSymEff, evidence = evidence)
    sumAbsen, sumTotal = 0
    
    for i,val in enumerate(mergeSymEff):
        arrProb = phi[val].values
        large = len(arrProb)
        sumAbsen += arrProb[large-1]*arrWeight[i]
        sumTotal += 1*arrWeight[i]
        
    fitness = sumAbsen / sumTotal
    return fitness

<b>Funcion para evaluar toda una población de individuos con la funcion de fitnes especificada</b>

In [ ]:
def evaluate_population(pathologyInd, fitness_fn):
    """ Evalua una poblacion de individuos con la funcion de fitness pasada """
    population = pathologyInd.population
    popsize    = len(population)
    for i in range(popsize):
        if population[i].fitness == -1:    # si el individuo no esta evaluado
            population[i].fitness = fitness_fn(pathologyInd, population[i].chromosome)

<b>Funcion que selecciona con el metodo de torneo un par de individuos de population para cruzamiento </b>

In [16]:
import random
valor = random.randint(2,3)
valor

3

In [ ]:
def select_parents_tourney(population):
    popsize = len(population)
    rival = []
    
    for i in range(random.randint(2,3)): 
        rival_index = random.randint(0,popsize-1)
        rival.append(rival_index)
    
    maxFitness = 0
    for index in rival:
        if (population[index].fitness > maxFitness):
            maxFitness = population[index].fitness 
    
    # Escoje el primer padre
    sumfitness = sum([indiv.fitness for indiv in population])  # suma total del fitness de la poblacion
    pickfitness = random.uniform(0, sumfitness)   # escoge un numero aleatorio entre 0 y sumfitness
    cumfitness = 0     # fitness acumulado
    for i in range(popsize):
        cumfitness += population[i].fitness
        if cumfitness > pickfitness: 
            iParent1 = i
            break
     
    # Escoje el segundo padre, desconsiderando el padre ya escogido
    sumfitness = sumfitness - population[iParent1].fitness # retira el fitness del padre ya escogido
    pickfitness = random.uniform(0, sumfitness)   # escoge un numero aleatorio entre 0 y sumfitness
    cumfitness = 0     # fitness acumulado
    for i in range(popsize):
        if i == iParent1: continue   # si es el primer padre 
        cumfitness += population[i].fitness
        if cumfitness > pickfitness: 
            iParent2 = i
            break        
    return (population[iParent1], population[iParent2])

<b>Funcion que selecciona sobrevivientes para la sgte generacion, dada la poblacion actual y poblacion de hijos </b>

In [ ]:
def select_survivors(population, offspring_population, numsurvivors):
    next_population = []
    population.extend(offspring_population) # une las dos poblaciones
    isurvivors = sorted(range(len(population)), key=lambda i: population[i].fitness, reverse=True)[:numsurvivors]
    for i in range(numsurvivors): next_population.append(population[isurvivors[i]])
    return next_population

<b>Algoritmo Genetico</b>   
Recibe una poblacion inicial, funcion de fitness, numero de generaciones (ngen) y taza de mutación (pmut)

In [ ]:
def genetic_algorithm(pathologyInd, fitness_fn, ngen=100, pmut=0.1):
    "Algoritmo Genetico "
    
    population = pathologyInd.population
    popsize    = len(population)
    evaluate_population(pathologyInd, fitness_fn)  # evalua la poblacion inicial
    ibest       = sorted(range(len(population)), key=lambda i: population[i].fitness, reverse=True)[:1]
    bestfitness = [population[ibest[0]].fitness]
    print("Poblacion inicial, best_fitness = {}".format(population[ibest[0]].fitness))
    
    for g in range(ngen):   # Por cada generacion
        
        ## Selecciona las parejas de padres para cruzamiento 
        mating_pool = []
        for i in range(int(popsize/2)): mating_pool.append(select_parents_roulette(population)) 
        
        ## Crea la poblacion descendencia cruzando las parejas del mating pool con Recombinación de 1 punto
        offspring_population = []
        for i in range(len(mating_pool)): 
            #offspring_population.extend( mating_pool[i][0].crossover_onepoint(mating_pool[i][1]) )
            offspring_population.extend( mating_pool[i][0].crossover_uniform(mating_pool[i][1]) )

        ## Aplica el operador de mutacion con probabilidad pmut en cada hijo generado
        for i in range(len(offspring_population)):
            if random.uniform(0, 1) < pmut: 
                offspring_population[i] = offspring_population[i].mutate_swap()
                #offspring_population[i] = offspring_population[i].mutate_position()
        
        ## Evalua la poblacion descendencia
        evaluate_population(offspring_population, fitness_fn)  # evalua la poblacion inicial
        
        ## Selecciona popsize individuos para la sgte. generación de la union de la pob. actual y  pob. descendencia
        population = select_survivors(population, offspring_population, popsize)

        ## Almacena la historia del fitness del mejor individuo
        ibest = sorted(range(len(population)), key=lambda i: population[i].fitness, reverse=True)[:1]
        bestfitness.append(population[ibest[0]].fitness)
        print("generacion {}, best_fitness = {}".format(g, population[ibest[0]].fitness))
    
    return population[ibest[0]], bestfitness  

 <b>Algoritmo de Busqueda Genetica para el problema de combinacion de medicamentos en adultos mayores</b>   

In [7]:
def genetic_search_medicines(pathologyName, background, medicines, symtopms, weights, effects, bayesianNetwork, fitness_fn, num_medicines=10, popsize=10, ngen=100, pmut=0.5):
    import random
    population = []

    ## Crea la poblacion inicial con cromosomas aleatorios
    for i in range(popsize):
        chromosome = [random.randint(0,1) for j in range(1,num_medicines+1)]
        random.shuffle(chromosome)
        population.append( Individual_medicines(chromosome) )
    
    pathologyInd = PathologyIndividual(pathologyName, background, medicines, symtopms, weights, effects, bayesianNetwork, population)
    ## Crea la poblacion inicial con los siguientes cromosomas    
    #chromosomes = [[1,3,1,3,1,3,1,3,1,3],
    #               [2,4,2,4,2,4,2,4,2,4],
    #               [3,5,3,5,3,5,3,5,3,5],
    #               [4,6,4,6,4,6,4,6,4,6],
    #               [5,7,5,7,5,7,5,7,5,7],
    #               [6,8,6,8,6,8,6,8,6,8],
    #               [7,9,7,9,7,9,7,9,7,9],
    #               [8,10,8,10,8,10,8,10,8,10],
    #               [9,1,9,1,9,1,9,1,9,1],
    #               [10,2,10,2,10,2,10,2,10,2] ] 
    
    #for i in range(popsize):
    #    population.append( Individual_nqueens(chromosomes[i]) )   
        
    ## llama al algoritmo genetico para encontrar una solucion al problema de las n reinas
    return genetic_algorithm(pathologyInd, fitness_fn, ngen, pmut)
        

# Probando el Algoritmo genetico

In [ ]:
import matplotlib.pyplot as plt

# busca solucion para el problema de 10 reinas. Usa 10 individuos, 100 generaciones y taza de mutación de 0.25
best_ind, bestfitness = genetic_search_nqueens(fitness_nqueens, 10, 10, 100, 0.3)
plt.plot(bestfitness)
plt.show()